- 사이트별로 직무내용이 너무 상이함, 구분을 지어주기가 쉽지 않음  
-> 학습에 의해 판단되므로 직무내용을 굳이 안넣어도 될 것 같음

In [46]:
import pandas as pd
import numpy as np
import re

### 원티드 데이터 전처리
- 직무명, 채용공고명, 자격요건, 우대사항, 혜택 및 복지, 기술스택 ・ 툴, 회사명, 출처  
 -> 회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지

In [47]:
wanted = pd.read_csv('./sample_data/wanted.csv')
wanted.head(2)

,직무명,채용공고명,해시태그,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,기술스택 ・ 툴,근무지역,회사명,회사분야,출처
0,웹 개발자,Python Developer (백엔드 주니어),#퇴사율5%이하#51~300명#설립4~9년#육아휴직#출산휴가#스톡옵션#건강검진#교육...,"- 간단소개\r\n혜움랩스는 세무,재무,경리 등 AI기반 비서형 금융 서비스를 Fa...",• 세무 알고리즘 자동화 설계 및 개발\r\n• 세무팀과 커뮤니케이션\r\n• AW...,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\r\n• Python(필수),...","• MSA 기반의 시스템 개발 경험이 있으신 분\r\n• AWS 를 활용한 개발, ...",# 몰입을 위한\r\n• 액면가 스톡옵션의 기회가 열려있습니다. (2023년 7월 ...,Github\r\nJavaScript\r\nTypeScript\r\nReact.js...,서울.한국,혜움랩스,"IT, 컨텐츠",https://www.wanted.co.kr/wd/167436
1,웹 개발자,프론트엔드 개발자 (주니어),#퇴사율5%이하#51~300명#설립4~9년#육아휴직#출산휴가#스톡옵션#건강검진#교육...,"- 간단소개\r\n혜움랩스는 세무,재무,경리 등 AI기반 비서형 금융 서비스를 Fa...",• 세무 관련 서비스 및 플랫폼의 프론트엔드 에플리케이션 개발\r\n• 요구사항 분...,• HTML / CSS / JavaScript 에 대한 이해\r\n• 컴퓨터 사이언...,"• Vue.js, React.js, Angular 등 SPA 개발 경험이 있는 분\...",# 몰입을 위한\r\n• 액면가 스톡옵션의 기회가 열려있습니다. (2023년 7월 ...,Github\r\nJavaScript\r\nTypeScript\r\nReact.js...,서울.한국,혜움랩스,"IT, 컨텐츠",https://www.wanted.co.kr/wd/167433


- column확인 후 필요없는 column제거
- 정해진 frame의 column_name으로 변경
- ```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지```

In [48]:
wanted.count()

직무명         8095
채용공고명       8074
해시태그        8090
회사소개        8089
주요업무        8090
자격요건        8084
우대사항        7888
혜택 및 복지     8078
기술스택 ・ 툴    5838
근무지역        8090
회사명         8090
회사분야        8090
출처          8095
dtype: int64

- 우대사항, 기술스택 부분에 nan값 존재  
-> 행별로 단어를 체크하는 관점에서 크게문제가 안될 것 같기도 함

In [49]:
wanted.columns

Index(['직무명', '채용공고명', '해시태그', '회사소개', '주요업무', '자격요건', '우대사항', '혜택 및 복지',
       '기술스택 ・ 툴', '근무지역', '회사명', '회사분야', '출처'],
      dtype='object')

In [50]:
wanted = wanted.drop(['해시태그','근무지역','회사분야','회사소개'],axis=1)
wanted.columns

Index(['직무명', '채용공고명', '주요업무', '자격요건', '우대사항', '혜택 및 복지', '기술스택 ・ 툴', '회사명',
       '출처'],
      dtype='object')

In [51]:
wanted.rename(columns={'직무명':'직무내용','채용공고명':'공고명','기술스택 ・ 툴':'기술 스택', '혜택 및 복지':'복지','출처':'출처 URL','우대사항':'우대조건',}, inplace=True)
wanted.columns

Index(['직무내용', '공고명', '주요업무', '자격요건', '우대조건', '복지', '기술 스택', '회사명', '출처 URL'], dtype='object')

- 지원기간은 이야기 해봐야 할 듯, 경력조건은 자격요건의 내용에서 추출

### wanted 패턴
- 경력 n년 이상
- 경력 n년이상
- n~k년
- n년~k년

In [52]:
def extract_years(string, patterns):
    for pattern in patterns:
        match = re.search(pattern, string)
        if match:
            if '~' in pattern:
                start_year = match.group(1)
                end_year = match.group(2)
                return f"{start_year}~{end_year}년"
            else:
                return f'{match.group(1)}년 이상'
    
    return "경력 무관"

In [53]:
max_count = wanted['자격요건'].count()
wanted['경력조건'] = None
patterns = [r'(\d+)년 이상', r'(\d+)년이상', r'(\d+)~(\d+)년', r'(\d+)년~(\d+)년']

for i in range(0, 50):
    wanted['경력조건'][i] = extract_years(wanted['자격요건'][i], patterns)

#wanted['경력조건'][:50]

- ```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지```

In [54]:
new_order = ['회사명', '지원기간', '공고명', '직무내용', '자격요건', '우대조건', '경력조건', '기술 스택', '출처 URL', '복지']
wanted = wanted.reindex(columns=new_order)
wanted.head(2)

,회사명,지원기간,공고명,직무내용,자격요건,우대조건,경력조건,기술 스택,출처 URL,복지
0,혜움랩스,NaN,Python Developer (백엔드 주니어),웹 개발자,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\r\n• Python(필수),...","• MSA 기반의 시스템 개발 경험이 있으신 분\r\n• AWS 를 활용한 개발, ...",2년 이상,Github\r\nJavaScript\r\nTypeScript\r\nReact.js...,https://www.wanted.co.kr/wd/167436,# 몰입을 위한\r\n• 액면가 스톡옵션의 기회가 열려있습니다. (2023년 7월 ...
1,혜움랩스,NaN,프론트엔드 개발자 (주니어),웹 개발자,• HTML / CSS / JavaScript 에 대한 이해\r\n• 컴퓨터 사이언...,"• Vue.js, React.js, Angular 등 SPA 개발 경험이 있는 분\...",경력 무관,Github\r\nJavaScript\r\nTypeScript\r\nReact.js...,https://www.wanted.co.kr/wd/167433,# 몰입을 위한\r\n• 액면가 스톡옵션의 기회가 열려있습니다. (2023년 7월 ...


### 잡플래닛 데이터 전처리

In [55]:
jobplanet = pd.read_csv('./sample_data/jobplanet.csv')
jobplanet.head(2)

,href,title,company_name,job_location,job_classification,due_date,work_experience,skills,company_intro,main_task,qualification,preference,procedure,benefits,job_location_detail,detail_description
0,https://www.jobplanet.co.kr/job/search?posting...,DB 관리 (카드IS 3팀),롯데정보통신(주),서울,데이터 엔지니어,2023.07.03 D-7,3년 이상,"SQL, MYSQL, DB, 데이터베이스","오늘을 새롭게, 내일을 이롭게. 롯데\r\n\r\n롯데와 함께 비전을 공유할 수 있...",ㆍSQL 튜닝\r\nㆍDatabase Admin\r\nㆍDB Performance 튜닝,ㆍ경력 3년 이상\r\nㆍOracle Database 운영/백업/모니터링/장애처리 ...,ㆍSQL 튜닝\r\nㆍOGG 솔루션 운영 경험\r\nㆍ금융권 대용량 DB시스템 운영...,지원서접수 > 서류전형 > 직무테스트/인성진단(온라인) > 면접전형 > 평판조회 >...,ㆍ복지포인트 및 경조지원\r\nㆍ건강검진\r\nㆍ학자금/교육지원 등 다양한 복리후생...,서울 금천구 가산동 533-2 롯데센터 15층,"<section class=""recruitment-detail""><div class..."
1,https://www.jobplanet.co.kr/job/search?posting...,DB & 데이터 Engineer,(주)비트나인,서울,데이터 엔지니어,2023.06.26 오늘 마감,경력무관,"데이터, DB, rdbms, 데이터엔지니어",비트나인에서 엔터프라이즈 사업부 영업대표 담당자를 모집합니다.\r\n\r\n2013...,"· 당사 DBMS제품 설치구축, 트러블슈팅, 튜닝 기술지원\r\n· RDBMS(Ag...",· 학력 : 4년재 대학 졸업 이상\r\n· 경력 : 해당 DBMS 분야의 엔지니어...,· DBMS에 대한 애정과 역량을 발휘하고 싶은 경력자\r\n· DBMS QA(DB...,NaN,"• 복지포인트를 지원해줘요(가족식비,여가비,취미활동비 등 연240만원 수준)\r\n...","서울 강남구 테헤란로 516, 4층","<section class=""recruitment-detail""><div class..."


- column확인 후 필요없는 column제거
- 정해진 frame의 column_name으로 변경
- ```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지```

In [56]:
jobplanet.columns

Index(['href', 'title', 'company_name', 'job_location', 'job_classification',
       'due_date', 'work_experience', 'skills', 'company_intro', 'main_task',
       'qualification', 'preference', 'procedure', 'benefits',
       'job_location_detail', 'detail_description'],
      dtype='object')

In [57]:
jobplanet['detail_description']

0      <section class="recruitment-detail"><div class...
1      <section class="recruitment-detail"><div class...
2      <section class="recruitment-detail"><div class...
3      <section class="recruitment-detail"><div class...
4      <section class="recruitment-detail"><div class...
                             ...                        
103    <section class="recruitment-detail"><div class...
104    <section class="recruitment-detail"><div class...
105    <section class="recruitment-detail"><div class...
106    <section class="recruitment-detail"><div class...
107    <section class="recruitment-detail"><div class...
Name: detail_description, Length: 108, dtype: object

In [58]:
jobplanet = jobplanet.drop(['job_location','company_intro','procedure','job_location_detail'],axis=1)
jobplanet.columns

Index(['href', 'title', 'company_name', 'job_classification', 'due_date',
       'work_experience', 'skills', 'main_task', 'qualification', 'preference',
       'benefits', 'detail_description'],
      dtype='object')

In [59]:
jobplanet.rename(columns={'href': '출처 URL', 
                          'title': '공고명',
                          'company_name':'회사명',
                          'job_classification':'직무내용',
                          'due_date':'지원기간',
                          'work_experience':'경력조건',
                          'skills':'기술 스택',
                          'main_task':'주요 업무',
                          'qualification':'자격요건',
                          'preference':'우대조건',
                          'benefits':'복지'
                          }, inplace=True)
jobplanet.columns

Index(['출처 URL', '공고명', '회사명', '직무내용', '지원기간', '경력조건', '기술 스택', '주요 업무',
       '자격요건', '우대조건', '복지', 'detail_description'],
      dtype='object')

In [60]:
new_order = ['회사명', '지원기간', '공고명', '직무내용', '자격요건', '우대조건', '경력조건', '기술 스택', '출처 URL', '복지']
jobplanet = jobplanet.reindex(columns=new_order)
jobplanet.head(2)

,회사명,지원기간,공고명,직무내용,자격요건,우대조건,경력조건,기술 스택,출처 URL,복지
0,롯데정보통신(주),2023.07.03 D-7,DB 관리 (카드IS 3팀),데이터 엔지니어,ㆍ경력 3년 이상\r\nㆍOracle Database 운영/백업/모니터링/장애처리 ...,ㆍSQL 튜닝\r\nㆍOGG 솔루션 운영 경험\r\nㆍ금융권 대용량 DB시스템 운영...,3년 이상,"SQL, MYSQL, DB, 데이터베이스",https://www.jobplanet.co.kr/job/search?posting...,ㆍ복지포인트 및 경조지원\r\nㆍ건강검진\r\nㆍ학자금/교육지원 등 다양한 복리후생...
1,(주)비트나인,2023.06.26 오늘 마감,DB & 데이터 Engineer,데이터 엔지니어,· 학력 : 4년재 대학 졸업 이상\r\n· 경력 : 해당 DBMS 분야의 엔지니어...,· DBMS에 대한 애정과 역량을 발휘하고 싶은 경력자\r\n· DBMS QA(DB...,경력무관,"데이터, DB, rdbms, 데이터엔지니어",https://www.jobplanet.co.kr/job/search?posting...,"• 복지포인트를 지원해줘요(가족식비,여가비,취미활동비 등 연240만원 수준)\r\n..."


In [62]:
jobplanet['지원기간'] = jobplanet['지원기간'].str.split('\xa0').str[0]

In [63]:
jobplanet['지원기간'].unique()

array(['2023.07.03', '2023.06.26', '2023.07.21', '2023.06.30',
       '2023.09.20', '2023.09.26', '2023.09.19', '2023.07.14',
       '2023.09.15', '2023.09.14', '2023.09.13', '2023.09.09',
       '2023.07.09', '2023.09.07', '2023.07.04', '2023.08.31',
       '2023.08.25', '2023.08.26', '2023.08.24', '2023.08.23',
       '2023.08.29', '2023.08.19', '2023.08.18', '2023.08.17',
       '2023.08.13', '2023.08.16', '2023.08.21', '2023.08.09',
       '2023.08.22', '2023.08.27', '2023.08.15', '2023.07.12',
       '2023.10.29', '2023.07.17', '2023.08.02', '2023.07.11',
       '2023.09.24', '2023.09.02', '2023.09.06', '2023.07.13',
       '2023.07.28', '2023.07.18', '2023.07.19', '2023.09.21',
       '2023.09.22', '2023.07.01', '2023.08.10', '2023.09.11',
       '2023.08.14', '2023.07.31', '2023.09.27'], dtype=object)

In [64]:
result = pd.concat([wanted, jobplanet], axis=0)
result.count()

회사명       8198
지원기간       108
공고명       8182
직무내용      8203
자격요건      8192
우대조건      7994
경력조건       158
기술 스택     5946
출처 URL    8203
복지        8185
dtype: int64

### 점핏 데이터 전처리

- 크롤링 문제 확인으로 직무내용 별로 csv 파일 생성 후 concat
- 지원기간과 경력조건 data만 맞추기

In [35]:
jumpit = pd.DataFrame()

for i in range(1, 22):
    if i == 14:
        continue
    df = pd.read_csv(f'./jumpit_data/크롤링_점핏_{i}.csv')
    jumpit = pd.concat([jumpit, df])

In [36]:
jumpit.head(10)

,회사명,지원기간,공고명,직무내용,자격요건,우대조건,경력조건,기술 스택,출처 URL,복지
0,미디어로그,마감일\n상시,미디어로그(LG그룹) 웹 하드 3.0 SM 경력 채용,서버/백엔드 개발자,"• Java, Spring, Spring Boot, JSP 개발 스킬 필수\n• O...","• LG U+ SI/SM 업무 경험\n• 서버 간 연동 처리 경험 (restful,...",경력\n경력 10~15년,Java\nJSP\nSpring Boot\nMariaDB\nOracle\nLinux,https://www.jumpit.co.kr/position/16607,• 통신비 지원\n- U+유모바일 음성무제한 요금제 지원\n\n• 자녀 학비 지원\...
1,지바이크,마감일\n상시,[지쿠터] PHP / Laravel,서버/백엔드 개발자,• 해당 경력 3년 이상\n• Laravel 사용 경험이 있으신 분\n• 코드리뷰에...,• 대규모 트래픽 서비스 환경에서 개발 및 운영 경험이 있으신 분\n• 대용량 배치...,경력\n경력 3~10년,Laravel\nApache HTTP Server\nREST API\nPHP\nRD...,https://www.jumpit.co.kr/position/1583,• 영어이름으로 실현하는 수평적 조직문화!\n 지쿠터엔 직위 직급 없이 영문 닉네임...
2,데이터비,마감일\n상시,백엔드 엔지니어 (NodeJS),서버/백엔드 개발자,• 결제 기능 개발 경험\n• AdonisJS 프레임워크를 활용한 개발 경험\n• ...,• 데이터와 소프트웨어 기술이 사람간의 의사소통을 더 원활하게 도울 수 있다고 믿는...,경력\n신입~10년,Node.js\nAWS\nPython\nAdonisJS,https://www.jumpit.co.kr/position/16273,"[보상 및 근무 조건]\n• 머신러닝, 딥러닝 인프라(GPU 등)에 대한 아낌없는 ..."
3,미디어로그,마감일\n상시,미디어로그(LG그룹) 풀스택 개발자 (사내 업무 시스템),서버/백엔드 개발자,• Java / Spring framework 개발 스킬 필수 (3년 차 이상)\n...,• LINUX 경험\n• 컴퓨터 공학 관련 전공 또는 관련 학위,경력\n경력 5~10년,Java\nSpring Framework\nNoSql\nLinux,https://www.jumpit.co.kr/position/16187,• 통신비 지원\n- U+유모바일 음성무제한 요금제 지원\n\n• 자녀 학비 지원\...
4,내몸랩,마감일\n상시,"백엔드, 서버 개발자(경력 3년 이상)",서버/백엔드 개발자,• Backend 개발 경력 3년 이상 혹은 그에 준하는 실력을 보유한 분\n• J...,• Web RTC등의 스트리밍 서비스 개발 경험이 있는 분\n• 네트워크/앱 보안에...,경력\n경력 3~6년,JavaScript\nSpring Data JPA\nJava,https://www.jumpit.co.kr/position/11720,"• 원거리 출퇴근자 재택 근무 및 선택적 탄력근무제\n• 4대 보험가입, 근로기준법..."
5,포스트보이저영업소,마감일\n상시,Java/ Kotlin 개발자,서버/백엔드 개발자,• Java 또는 Kotlin 개발 경력 5년 이상\n• Linux에서 웹 서비스 ...,"• MSA 개발/운영 경험\n• Go, Rust, Koltin 개발 경험\n• 대용...",경력\n경력 5~10년,Java\nLinux\nKotlin\nREST API\nAWS\nGCP\nGit\n...,https://www.jumpit.co.kr/position/13204,• 산전/산후휴가 / 육아휴가\n• 유급휴가 / 경조휴가 / 기념일 반차\n• 법정...
6,위허들링,마감일\n상시,위잇딜라이트 백엔드 개발자,서버/백엔드 개발자,• Python 기반 Restful API(Django & Django Rest F...,• 아래 경험들을 해보셨다면 위잇딜라이트는 우대합니다.\n - Django & Dj...,경력\n경력 3~10년,Python\nDjango\nPostgreSQL\nAWS\nGit\nGitLab\n...,https://www.jumpit.co.kr/position/4713,"• 주 35시간, 하루 7시간 근무 (표준근무시간 10:00-18:00)\n• 근로..."
7,더블미디어,마감일\n상시,백엔드 개발자 모집,서버/백엔드 개발자,"• (백엔드 담당 A) TypeScript, Node,js, NestJS, ORM에...",• OAuth2 서버 구축 및 서비스 적용 경험이 있으신 분\n• CICD 에 능숙...,경력\n경력 2~10년,TypeScript\nNestJS\nGolang,https://www.jumpit.co.kr/position/14733,"• 사내 카페 및 간식코너 (라면,과자,커피,음료 등)\n• 성과급 제도\n• 각종..."
8,로보코리아,마감일\n상시,백엔드 엔지니어,서버/백엔드 개발자,• 새로운 기술 및 트렌드에 대해 끊임없이 관심을 갖고 이를 활용 하시는 분\n• ...,"• 글로벌 서비스를 위한 인프라 아키텍처 설계, 개발 경험이 있으신 분\n• 애자일...",경력\n경력 3~10년,TypeScript\nNestJS\nMongoDB\nNoSql\nMSA\nDatad...,https://www.jumpit.co.kr/position/7827,1. 자율과 효율의 근무 환경을 제공합니다.\n• 오전 09:00 - 10:30 유...
9,렌딧,마감일\n2023-07-31,Senior Back-end Engineer,서버/백엔드 개발자,"• 관련 업무 경력 3년 이상\n• Kotlin, Java 기반 웹 어플리케이션 프...","• AWS를 활용한 서비스 개발/운영 경험이 있으신 분\n• 알고리즘, 성능최적화,...",경력\n경력 3~20년,Kotlin\nJava\nSpring Boot\nSpring Framework\nA...,https://www.jumpit.co.kr/position/14158,"[개발자의 성장을 돕는 활동]\n• 개인성장을 위한 도서 구입, 온라인 강의 수강 ..."


In [37]:
jumpit.count()

회사명       3190
지원기간      3190
공고명       3190
직무내용      3190
자격요건      3190
우대조건      3190
경력조건      3190
기술 스택     3190
출처 URL    3190
복지        3190
dtype: int64

- 근무지역, 지도보기 등 오류내용 껴있는 부분 노가다 수정
- 마감일 단어제거, ```'-' -> '.'```형태 변화

In [38]:
#지원기간 column 오류내용 수정

jumpit.loc[jumpit['지원기간'] == '근무지역\n서울특별시 성동구 하왕십리동 286-2, 2층\n지도보기·주소복사'] = '상시'
jumpit.loc[jumpit['지원기간'] == '근무지역\n본점 : 서울 영등포구 은행로3 901호, 연구소 : 대구 수성구 알파시티1로170 DIP 310호\n지도보기·주소복사'] = '상시'
jumpit.loc[jumpit['지원기간'] == '근무지역\n부산시 해운대구 우동 센텀중앙로48 에이스하이테크1212 또는 서울 강남 지사 근무 (구직자 선택사항)\n지도보기·주소복사'] = '상시'
jumpit.loc[jumpit['지원기간'] == '근무지역\n서울시 구로구 디지털로26길 111 제이앤케이디지털타워 612호\n지도보기·주소복사'] = '상시'
jumpit.loc[jumpit['지원기간'] == '근무지역\n서울 마포구 성암로 330, 621호\n지도보기·주소복사'] = '상시'
jumpit.loc[jumpit['지원기간'] == '근무지역\n서울시 금천구 가산디지털1로 145 에이스하이엔드타워3차 1704호\n지도보기·주소복사'] = '상시'
jumpit.loc[jumpit['지원기간'] == '근무지역\n부산시 해운대구 우동 센텀중앙로48 에이스하이테크1212 또는 서울 강남 근무 (구직자 선택사항)\n지도보기·주소복사'] = '상시'
jumpit.loc[jumpit['지원기간'] == '근무지역\n서울시 강남구 남부순환로 2645, 501-5호\n지도보기·주소복사'] = '상시'
jumpit.loc[jumpit['지원기간'] == '근무지역\n대구광역시 달서구 성서공단북로 65길29\n지도보기·주소복사'] = '상시'
jumpit.loc[jumpit['지원기간'] == '근무지역\n서울 종로구 사직로 130, 602호\n지도보기·주소복사'] = '상시'

In [39]:
# 주소검색 확인용 코드

jumpit.loc[jumpit['지원기간'] == '근무지역\n서울 종로구 사직로 130, 602호\n지도보기·주소복사']

,회사명,지원기간,공고명,직무내용,자격요건,우대조건,경력조건,기술 스택,출처 URL,복지


In [40]:
#마감일\n 삭제와 '-' -> '.' replace

jumpit['지원기간'] = jumpit['지원기간'].str.replace('마감일\n', '').str.replace('-','.')

In [41]:
jumpit['지원기간'].unique()

array(['상시', '2023.07.31', '2023.08.31', '2023.08.01', '2023.07.21',
       '2023.08.16', '2023.07.24', '2023.08.04', '2023.07.28',
       '2023.07.26', '2023.08.03', '2023.07.23', '2023.07.17',
       '2023.12.31', '2023.09.07', '2023.08.07', '2023.07.29',
       '2023.07.16', '2023.09.30', '2023.09.13', '2023.11.30',
       '2023.07.20', '2023.07.30', '2023.08.11', '2023.07.18',
       '2023.08.21', '2023.09.12', '2023.08.06', '2023.08.02',
       '2023.08.12', '2023.10.09', '2023.10.03', '2023.08.22',
       '2023.09.08', '2023.08.23', '2023.08.17', '2023.08.09',
       '2023.09.11', '2023.08.15', '2023.08.08', '2023.07.25',
       '2023.08.28', '2023.08.10', '2023.07.19', '2023.08.13',
       '2023.10.02', '2023.09.29', '2023.09.27', '2023.12.01',
       '2023.08.05', '2023.09.06'], dtype=object)

- 경력 조건 부분 ```경력\n``` 제거
- 신입~n년 부분 1~n 년으로 변경

In [42]:
jumpit['경력조건'] = jumpit['경력조건'].str.replace('경력\n', '')
jumpit['경력조건'] = jumpit['경력조건'].str.replace('신입~', '경력 1~')

In [43]:
jumpit['경력조건'].unique()

array(['경력 10~15년', '경력 3~10년', '경력 1~10년', '경력 5~10년', '경력 3~6년',
       '경력 2~10년', '경력 3~20년', '경력 1~7년', '경력 2~6년', '경력 2~20년',
       '경력 5~20년', '경력 5~12년', '경력 7~15년', '경력 10~20년', '경력 5~15년',
       '경력 1~5년', '경력 3~7년', '경력 3~15년', '경력 1~3년', '경력 2~9년', '경력 3~5년',
       '경력 7~20년', '경력 1~2년', '경력 3~12년', '경력 4~6년', '경력 2~7년',
       '경력 1~20년', '경력 4~15년', '경력 7~10년', '경력 2~15년', '경력 4~10년',
       '경력 6~20년', '경력 1~15년', '경력 6~15년', '경력 10~18년', '경력 2~5년',
       '경력 3~8년', '경력 5~8년', '경력 4~11년', '경력 2~8년', '경력 6~16년',
       '경력 3~11년', '경력 8~15년', '경력 6~8년', '경력 2~3년', '경력 5~7년',
       '경력 2~12년', '경력 5~13년', '경력 3~9년', '경력 4~7년', '경력 1~8년',
       '경력 7~25년', '신입', '경력 10~14년', '경력 3~4년', '경력 8~13년', '경력 4~20년',
       '경력 1~4년', '경력 6~17년', '경력 2~16년', '경력 4~12년', '상시', '경력 4~9년',
       '경력 8~10년', '경력 1~13년', '경력 4~8년', '경력 3~19년', '경력 5~14년',
       '경력 5~11년', '경력 6~10년', '경력 1~16년', '경력 8~20년', '경력 5~9년',
       '경력 6~9년', '경력 3~13년', '경력 3~30년', '경력 1~9년', '경력 4~1

In [44]:
jumpit.head(10)

,회사명,지원기간,공고명,직무내용,자격요건,우대조건,경력조건,기술 스택,출처 URL,복지
0,미디어로그,상시,미디어로그(LG그룹) 웹 하드 3.0 SM 경력 채용,서버/백엔드 개발자,"• Java, Spring, Spring Boot, JSP 개발 스킬 필수\n• O...","• LG U+ SI/SM 업무 경험\n• 서버 간 연동 처리 경험 (restful,...",경력 10~15년,Java\nJSP\nSpring Boot\nMariaDB\nOracle\nLinux,https://www.jumpit.co.kr/position/16607,• 통신비 지원\n- U+유모바일 음성무제한 요금제 지원\n\n• 자녀 학비 지원\...
1,지바이크,상시,[지쿠터] PHP / Laravel,서버/백엔드 개발자,• 해당 경력 3년 이상\n• Laravel 사용 경험이 있으신 분\n• 코드리뷰에...,• 대규모 트래픽 서비스 환경에서 개발 및 운영 경험이 있으신 분\n• 대용량 배치...,경력 3~10년,Laravel\nApache HTTP Server\nREST API\nPHP\nRD...,https://www.jumpit.co.kr/position/1583,• 영어이름으로 실현하는 수평적 조직문화!\n 지쿠터엔 직위 직급 없이 영문 닉네임...
2,데이터비,상시,백엔드 엔지니어 (NodeJS),서버/백엔드 개발자,• 결제 기능 개발 경험\n• AdonisJS 프레임워크를 활용한 개발 경험\n• ...,• 데이터와 소프트웨어 기술이 사람간의 의사소통을 더 원활하게 도울 수 있다고 믿는...,경력 1~10년,Node.js\nAWS\nPython\nAdonisJS,https://www.jumpit.co.kr/position/16273,"[보상 및 근무 조건]\n• 머신러닝, 딥러닝 인프라(GPU 등)에 대한 아낌없는 ..."
3,미디어로그,상시,미디어로그(LG그룹) 풀스택 개발자 (사내 업무 시스템),서버/백엔드 개발자,• Java / Spring framework 개발 스킬 필수 (3년 차 이상)\n...,• LINUX 경험\n• 컴퓨터 공학 관련 전공 또는 관련 학위,경력 5~10년,Java\nSpring Framework\nNoSql\nLinux,https://www.jumpit.co.kr/position/16187,• 통신비 지원\n- U+유모바일 음성무제한 요금제 지원\n\n• 자녀 학비 지원\...
4,내몸랩,상시,"백엔드, 서버 개발자(경력 3년 이상)",서버/백엔드 개발자,• Backend 개발 경력 3년 이상 혹은 그에 준하는 실력을 보유한 분\n• J...,• Web RTC등의 스트리밍 서비스 개발 경험이 있는 분\n• 네트워크/앱 보안에...,경력 3~6년,JavaScript\nSpring Data JPA\nJava,https://www.jumpit.co.kr/position/11720,"• 원거리 출퇴근자 재택 근무 및 선택적 탄력근무제\n• 4대 보험가입, 근로기준법..."
5,포스트보이저영업소,상시,Java/ Kotlin 개발자,서버/백엔드 개발자,• Java 또는 Kotlin 개발 경력 5년 이상\n• Linux에서 웹 서비스 ...,"• MSA 개발/운영 경험\n• Go, Rust, Koltin 개발 경험\n• 대용...",경력 5~10년,Java\nLinux\nKotlin\nREST API\nAWS\nGCP\nGit\n...,https://www.jumpit.co.kr/position/13204,• 산전/산후휴가 / 육아휴가\n• 유급휴가 / 경조휴가 / 기념일 반차\n• 법정...
6,위허들링,상시,위잇딜라이트 백엔드 개발자,서버/백엔드 개발자,• Python 기반 Restful API(Django & Django Rest F...,• 아래 경험들을 해보셨다면 위잇딜라이트는 우대합니다.\n - Django & Dj...,경력 3~10년,Python\nDjango\nPostgreSQL\nAWS\nGit\nGitLab\n...,https://www.jumpit.co.kr/position/4713,"• 주 35시간, 하루 7시간 근무 (표준근무시간 10:00-18:00)\n• 근로..."
7,더블미디어,상시,백엔드 개발자 모집,서버/백엔드 개발자,"• (백엔드 담당 A) TypeScript, Node,js, NestJS, ORM에...",• OAuth2 서버 구축 및 서비스 적용 경험이 있으신 분\n• CICD 에 능숙...,경력 2~10년,TypeScript\nNestJS\nGolang,https://www.jumpit.co.kr/position/14733,"• 사내 카페 및 간식코너 (라면,과자,커피,음료 등)\n• 성과급 제도\n• 각종..."
8,로보코리아,상시,백엔드 엔지니어,서버/백엔드 개발자,• 새로운 기술 및 트렌드에 대해 끊임없이 관심을 갖고 이를 활용 하시는 분\n• ...,"• 글로벌 서비스를 위한 인프라 아키텍처 설계, 개발 경험이 있으신 분\n• 애자일...",경력 3~10년,TypeScript\nNestJS\nMongoDB\nNoSql\nMSA\nDatad...,https://www.jumpit.co.kr/position/7827,1. 자율과 효율의 근무 환경을 제공합니다.\n• 오전 09:00 - 10:30 유...
9,렌딧,2023.07.31,Senior Back-end Engineer,서버/백엔드 개발자,"• 관련 업무 경력 3년 이상\n• Kotlin, Java 기반 웹 어플리케이션 프...","• AWS를 활용한 서비스 개발/운영 경험이 있으신 분\n• 알고리즘, 성능최적화,...",경력 3~20년,Kotlin\nJava\nSpring Boot\nSpring Framework\nA...,https://www.jumpit.co.kr/position/14158,"[개발자의 성장을 돕는 활동]\n• 개인성장을 위한 도서 구입, 온라인 강의 수강 ..."


In [45]:
jumpit.to_csv('jumpit.csv',index = False)

- 전체 concat

In [65]:
result = pd.concat([result, jumpit], axis=0)
result.count()

회사명       11388
지원기간       3298
공고명       11372
직무내용      11393
자격요건      11382
우대조건      11184
경력조건       3348
기술 스택      9136
출처 URL    11393
복지        11375
dtype: int64

In [66]:
result.to_csv('result.csv',index = False)